Planning to alpha vantage to bring in the international data.

The alpha vantage key linked to sastra account : K3Y77VRIVPOWAO2G

In [1]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
api_key = "K3Y77VRIVPOWAO2G"
url = f'https://www.alphavantage.co/query?function=MARKET_STATUS&apikey={api_key}'
r = requests.get(url)# Corrected code
data = r.json()

print(data)

{'endpoint': 'Global Market Open & Close Status', 'markets': [{'market_type': 'Equity', 'region': 'United States', 'primary_exchanges': 'NASDAQ, NYSE, AMEX, BATS', 'local_open': '09:30', 'local_close': '16:15', 'current_status': 'open', 'notes': ''}, {'market_type': 'Equity', 'region': 'Canada', 'primary_exchanges': 'Toronto, Toronto Ventures', 'local_open': '09:30', 'local_close': '16:00', 'current_status': 'open', 'notes': ''}, {'market_type': 'Equity', 'region': 'United Kingdom', 'primary_exchanges': 'London', 'local_open': '08:00', 'local_close': '16:30', 'current_status': 'closed', 'notes': ''}, {'market_type': 'Equity', 'region': 'Germany', 'primary_exchanges': 'XETRA, Berlin, Frankfurt, Munich, Stuttgart', 'local_open': '08:00', 'local_close': '20:00', 'current_status': 'open', 'notes': ''}, {'market_type': 'Equity', 'region': 'France', 'primary_exchanges': 'Paris', 'local_open': '09:00', 'local_close': '17:30', 'current_status': 'closed', 'notes': ''}, {'market_type': 'Equity',

trying to get the required data directly from nse using this : https://www.nseindia.com/api/NextApi/apiClient?functionName=getReferenceRates&&type=null&&flag=CUR

In [13]:
import requests

url = "https://www.nseindia.com/api/NextApi/apiClient?functionName=getReferenceRates&&type=null&&flag=CUR"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json",
    "Referer": "https://www.nseindia.com/"
}

response = requests.get(url, headers=headers)
data = response.json()

In [14]:
import pprint

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(data)

{ 'data': { 'commoditySpotRates': None,
            'currencySpotRates': [ { 'currency': 'USD',
                                     'prevDayValue': '87.7414',
                                     'unit': '1',
                                     'value': '87.6559'},
                                   { 'currency': 'YEN',
                                     'prevDayValue': '59.4200',
                                     'unit': '100',
                                     'value': '59.5300'},
                                   { 'currency': 'GBP',
                                     'prevDayValue': '118.1081',
                                     'unit': '1',
                                     'value': '118.3822'},
                                   { 'currency': 'EURO',
                                     'prevDayValue': '102.0351',
                                     'unit': '1',
                                     'value': '102.0087'}],
            'intrestRates': None,
      

# Writing into an Excel sheet

In [17]:
import pandas as pd
import requests
import win32com.client as win32
import pythoncom
import time
from pathlib import Path
import pprint

class NseCurrencyUpdaterCOM:
    """
    Fetches currency spot rates from NSE and updates an Excel sheet.
    """
    def __init__(self, file_path: str, offset_rows: int = 5, offset_cols: int = 1):
        self.file_path = Path(file_path)
        self.offset_rows = offset_rows
        self.offset_cols = offset_cols
        self.headers = {
            "User-Agent": "Mozilla/5.0",
            "Accept": "application/json",
            "Referer": "https://www.nseindia.com/"
        }
        self.url = "https://www.nseindia.com/api/NextApi/apiClient?functionName=getReferenceRates&&type=null&&flag=CUR"

    def fetch_data(self):
        """
        Fetches currency spot rates from the NSE API.
        """
        try:
            session = requests.Session()
            response = session.get(self.url, headers=self.headers)
            response.raise_for_status()  # Raise an exception for bad status codes
            full_data = response.json()
            
            # Check if the expected data exists
            if 'data' not in full_data or 'currencySpotRates' not in full_data['data']:
                raise KeyError("API response is missing 'data' or 'currencySpotRates' key.")
            
            currency_data = full_data['data']['currencySpotRates']
            timestamp = full_data['data'].get('timeStamp')
            pp = pprint.PrettyPrinter(indent=2)
            pp.pprint(currency_data)
            return currency_data, timestamp

        except requests.exceptions.RequestException as e:
            raise Exception(f"Failed to fetch data from NSE. Error: {e}")
        except KeyError as e:
            raise Exception(f"Failed to parse data from NSE. Missing key: {e}")

    def prepare_dataframe(self, data):
        """
        Prepares a pandas DataFrame from the fetched currency data.
        """
        df = pd.DataFrame(data)
        # Rename columns for clarity, using title case
        df.rename(columns={
            'currency': 'Currency',
            'value': 'Value',
            'prevDayValue': 'Prev. Day Value',
            'unit': 'Unit'
        }, inplace=True)
        # Reorder columns to a more logical flow
        column_order = ['Currency', 'Unit', 'Value', 'Prev. Day Value']
        df = df[column_order]
        return df

    def write_using_com(self, df: pd.DataFrame, timestamp: str):
        """
        Writes the DataFrame to an Excel sheet using COM automation, starting at an offset.
        """
        def safe_set_attr(obj_getter, attr, value, retries=5, delay=0.5):
            for i in range(retries):
                try:
                    obj = obj_getter()
                    setattr(obj, attr, value)
                    return
                except Exception as e:
                    if '0x800ac472' in str(e): # Busy/RPC_S_CALLPENDING error
                        time.sleep(delay)
                    else:
                        raise
            print(f"Failed to set {attr} after {retries} attempts.")

        pythoncom.CoInitialize()
        excel = win32.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True
        excel.DisplayAlerts = False

        try:
            wb = None
            for open_wb in excel.Workbooks:
                if open_wb.FullName.lower() == str(self.file_path).lower():
                    wb = open_wb
                    break
            
            if wb is None:
                if self.file_path.exists():
                    wb = excel.Workbooks.Open(str(self.file_path))
                else:
                    wb = excel.Workbooks.Add()
                    wb.SaveAs(str(self.file_path))

            ws = wb.Worksheets(1)
            ws.Name = "Currency Spot Rates"

            try:
                if ws.ProtectContents:
                    ws.Unprotect()
            except Exception as e:
                print(f"Warning: Couldn't unprotect sheet: {e}")
            
            # Start clearing from the specified offset
            start_cell = ws.Cells(self.offset_rows, self.offset_cols)
            last_cell = ws.Cells(ws.Rows.Count, ws.Columns.Count)
            ws.Range(start_cell, last_cell).ClearContents()

            # Write timestamp with offset
            timestamp_cell = ws.Cells(self.offset_rows, self.offset_cols)
            safe_set_attr(lambda: timestamp_cell, "Value", f"Data fetched at: {timestamp}")
            
            # Write headers with offset
            headers = df.columns.tolist()
            for col_index, header in enumerate(headers, start=self.offset_cols):
                cell = ws.Cells(self.offset_rows + 2, col_index)
                safe_set_attr(lambda c=cell: c, "Value", header)
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)

            # Write data with offset
            for row_index, row in enumerate(df.itertuples(index=False), start=self.offset_rows + 3):
                for col_index, value in enumerate(row, start=self.offset_cols):
                    safe_set_attr(lambda r=row_index, c=col_index: ws.Cells(r, c), "Value", value)

            # AutoFit columns for better readability
            ws.Columns.AutoFit()

            print(f"Currency data updated successfully at {timestamp}.")

        except Exception as e:
            print(f"Error during Excel update: {e}")
            raise  # Re-raise the exception to signal a failure

        finally:
            excel.DisplayAlerts = True
            pythoncom.CoUninitialize()

    def run(self):
        """
        Executes the full process of fetching and writing data.
        """
        print("Fetching currency spot rates...")
        data, timestamp = self.fetch_data()
        print(f"Data fetched successfully at {timestamp}. Preparing DataFrame...")
        df = self.prepare_dataframe(data)
        print("DataFrame prepared. Writing to Excel...")
        self.write_using_com(df, timestamp)


In [18]:
if __name__ == '__main__':
    # Specify the Excel file path and the desired offset
    excel_file = Path.cwd() / "nse_currency_data.xlsx"
    offset_rows = 5  
    offset_cols = 2  

    updater = NseCurrencyUpdaterCOM(
        file_path=str(excel_file),
        offset_rows=offset_rows,
        offset_cols=offset_cols
    )
    updater.run()

Fetching currency spot rates...
[ { 'currency': 'USD',
    'prevDayValue': '87.7414',
    'unit': '1',
    'value': '87.6559'},
  { 'currency': 'YEN',
    'prevDayValue': '59.4200',
    'unit': '100',
    'value': '59.5300'},
  { 'currency': 'GBP',
    'prevDayValue': '118.1081',
    'unit': '1',
    'value': '118.3822'},
  { 'currency': 'EURO',
    'prevDayValue': '102.0351',
    'unit': '1',
    'value': '102.0087'}]
Data fetched successfully at 28-Aug-2025 13:00:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
Currency data updated successfully at 28-Aug-2025 13:00:00.
